In [6]:
import time
import pandas as pd
import asyncio
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re
from concurrent.futures import ThreadPoolExecutor

chrome_options = Options()
chrome_options.add_argument('--headless')


In [2]:
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from bs4 import BeautifulSoup

# chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run Chrome in headless mode (without UI)
# driver = webdriver.Chrome(options=chrome_options)

# start_url = "https://lalafo.az/baku/ads/binqdi-qs-3-otaqli-90-kv-m-yeni-tmirli-id-102934449"
# driver.get(start_url)

# soup = BeautifulSoup(driver.page_source, "html.parser")

# try:
#     id = soup.select_one('div.about-ad-info__id span[style*="color:#737d9b"]').get_text()
#     link = soup.select_one('link[rel="canonical"]')['href']
#     update_date = soup.select_one('div.about-ad-info__date span:contains("Yenilənmə tarixi") + span').get_text()
#     create_date = soup.select_one('div.about-ad-info__date span:contains("Yaradılma vaxtı") + span').get_text()
#     price = soup.select_one('span.price').get_text()
#     currency = soup.select_one('span.currency').get_text()
#     user_name = soup.select_one('span.userName-text').get_text()
#     user_status = soup.select_one('p.LFParagraph.size-14.userStatus').get_text()
#     phone_number = [item.get_text() for item in soup.select('.phone-item span')]
#     address_description = soup.select_one('div.description__wrap p span').get_text()
#     address = soup.select_one('div.pro-item.address div.pro-item__title-wrap p').get_text()
#     region_addres = soup.select_one('li p.Paragraph.secondary:contains("Rayon:") + a.LinkText.primary-black.extra-small').get_text()
#     metro_station = soup.select_one('li p.Paragraph.secondary:contains("Metro stansiyası:") + a.LinkText.primary-black.extra-small').get_text()
#     official_address = soup.select_one('li p.Paragraph.secondary:contains("İnzibati rayonlar:") + a.LinkText.primary-black.extra-small').get_text()
#     impressions = soup.select_one('span.Caption.primary:contains("Göstərilmə")').get_text()
#     view_counts = soup.select_one('div.details-page__statistic-bar span.Caption.primary:contains("Baxış sayı")').get_text()
#     likes = soup.select_one('div.details-page__statistic-bar span.Caption.primary:contains("Bəyən")').get_text()
#     show_button = bool(soup.select_one('button.show-button'))
#     short_description = soup.select_one('h1.Heading.secondary-small').get_text()
#     count_of_rooms = soup.select_one('li p.Paragraph.secondary:contains("Otaqların sayı:") + a.LinkText.primary-black.extra-small').get_text()
#     area_of_flat = soup.select_one('li p.Paragraph.secondary:contains("Sahə (m2):") + p.Paragraph.secondary').get_text()
#     area_of_property = soup.select_one('li p.Paragraph.secondary:contains("Torpaq sahəsi (Sot):") + p.Paragraph.secondary').get_text()
#     floor = soup.select_one('li p.Paragraph.secondary:contains("Mərtəbələrin sayı:") + p.Paragraph.secondary').get_text()
#     repair_type = soup.select_one('ul.details-page__params li:nth-child(8) a').get_text()
#     flat_type = [item.get_text() for item in soup.select('ul.details-page__params li p:contains("Evin şəraiti:") + a.LinkText.primary-black.extra-small')]
#     communal_lines = [item.get_text() for item in soup.select('ul.details-page__params li:nth-child(9) a')]

#     yield {
#         "link": link,
#         "id": id,
#         "update_date": update_date,
#         "create_date": create_date,
#         "price": price,
#         "currency": currency,
#         "user_name": user_name,
#         "user_status": user_status,
#         "phone_number": phone_number,
#         "address_description": address_description,
#         "address": address,
#         "region_addres": region_addres,
#         "metro_station": metro_station,
#         "official_address": official_address,
#         "impressions": impressions,
#         "view_counts": view_counts,
#         "likes": likes,
#         "show_button": show_button,
#         "short_description": short_description,
#         "count_of_rooms": count_of_rooms,
#         "area_of_flat": area_of_flat,
#         "area_of_property": area_of_property,
#         "floor": floor,
#         "repair_type": repair_type,
#         "flat_type": flat_type,
#         "communal_lines": communal_lines
#     }
# except Exception as e:
#     # Log the exception or handle it as per your requirement.
#     # You can also yield an error item if needed.
#     self.logger.error(f"An error occurred while parsing: {str(e)}")
#     yield {
#         "link": start_url,
#         "id": None,
#         "update_date": None,
#         "create_date": None,
#         "price": None,
#         "currency": None,
#         "user_name": None,
#         "user_status": None,
#         "phone_number": None,
#         "address_description": None,
#         "address": None,
#         "region_addres": None,
#         "metro_station": None,
#         "official_address": None,
#         "impressions": None,
#         "view_counts": None,
#         "likes": None,
#         "show_button": None,
#         "short_description": None,
#         "count_of_rooms": None,
#         "area_of_flat": None,
#         "area_of_property": None,
#         "floor": None,
#         "repair_type": None,
#         "flat_type": None,
#         "communal_lines": None
#     }

# driver.quit()


In [20]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def click_show_buttons(driver, num_buttons):
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'show-button')))
    show_buttons = driver.find_elements(By.CLASS_NAME, 'show-button')

    for i in range(min(num_buttons, len(show_buttons))):
        show_buttons[i].click()
        time.sleep(1)  # Add a short delay to allow the content to load (optional)

if __name__ == "__main__":
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Run Chrome WebDriver in headless mode
    driver = webdriver.Chrome(options=chrome_options)
    driver.get("https://lalafo.az/baku/ads/baki-hovsan-qs-74-kv-m-3-otaqli-hovuzsuz-kombi-qaz-isiq-id-92773245")
    time.sleep(2)

    # Click on the first four "show" buttons
    num_buttons_to_click = 4
    click_show_buttons(driver, num_buttons_to_click)

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'phone-item')))
    updated_html = driver.page_source

    driver.quit()


In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def extract_phone_numbers(driver, url):
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'phone-item')))
    phone_items = driver.find_elements(By.CLASS_NAME, 'phone-item')
    phone_numbers = []

    for phone_item in phone_items:
        phone_number = phone_item.find_element(By.TAG_NAME, 'span').text.strip()
        phone_numbers.append(phone_number)

    return phone_numbers

if __name__ == "__main__":
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Run Chrome WebDriver in headless mode
    driver = webdriver.Chrome(options=chrome_options)

    url = "https://lalafo.az/baku/ads/baki-hovsan-qs-74-kv-m-3-otaqli-hovuzsuz-kombi-qaz-isiq-id-92773245"
    extracted_phone_numbers = extract_phone_numbers(driver, url)
    print(extracted_phone_numbers)

    driver.quit()


['+ 9947020230 xx', '+ 9945520230 xx']


In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def click_show_buttons(driver, num_buttons):
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'show-button')))
    show_buttons = driver.find_elements(By.CLASS_NAME, 'show-button')

    for i in range(min(num_buttons, len(show_buttons))):
        show_buttons[i].click()
        time.sleep(1)  # Add a short delay to allow the content to load (optional)

def extract_phone_numbers(driver):
    phone_numbers = []

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'phone-item')))
    phone_items = driver.find_elements(By.CLASS_NAME, 'phone-item')

    for phone_item in phone_items:
        phone_number = phone_item.find_element(By.TAG_NAME, 'span').text.strip()
        phone_numbers.append(phone_number)

    return phone_numbers

if __name__ == "__main__":
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Run Chrome WebDriver in headless mode
    driver = webdriver.Chrome(options=chrome_options)

    url = "https://lalafo.az/baku/ads/baki-hovsan-qs-74-kv-m-3-otaqli-hovuzsuz-kombi-qaz-isiq-id-92773245"
    driver.get(url)
    time.sleep(2)

    # Click on the first four "show" buttons
    num_buttons_to_click = 4
    click_show_buttons(driver, num_buttons_to_click)

    extracted_phone_numbers = extract_phone_numbers(driver)
    print(extracted_phone_numbers)

    driver.quit()


['+ 994702023062', '+ 994552023062']


In [25]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage you want to scrape
url = "https://lalafo.az/azerbaijan/nedvizhimost"  # Replace with the actual URL

# Send an HTTP GET request to the URL
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")
# Use BeautifulSoup to find all <a> tags with an 'href' attribute
all_links = soup.find_all("a", href=True)

# Loop through the links and extract the 'href' attribute
for link in all_links:
    href = link["href"]
    print(href)


/khyrdalan/ads/13-sot-tikinti-mulkiyytci-bldiyy-id-93990191
/khyrdalan/ads/13-sot-tikinti-mulkiyytci-bldiyy-id-93990191
/user/2657448
/user/2657448
/baku/ads/50-kv-m-1-otaqli-id-77935482
/baku/ads/50-kv-m-1-otaqli-id-77935482
/ghb
/ghb
/baku/ads/tcili-elan-id-84402679
/baku/ads/tcili-elan-id-84402679
/user/13916368
/user/13916368
/baku/ads/xaricden-gelen-ve-yerli-qonaqlarimiz-ucun-gunluk-ve-yari-gunluk-evler-id-80995625
/baku/ads/xaricden-gelen-ve-yerli-qonaqlarimiz-ucun-gunluk-ve-yari-gunluk-evler-id-80995625
/user/12150524
/user/12150524
/khyrdalan/ads/40kv-yeni-tikilinin-1mertebesi-yola-yaxin-yerdir-real-alicilar-zeng-id-82468566
/khyrdalan/ads/40kv-yeni-tikilinin-1mertebesi-yola-yaxin-yerdir-real-alicilar-zeng-id-82468566
/user/13477695
/user/13477695
/baku/ads/salam-buzovna-qsbsind-cox-gozl-v-qdim-razid-id-83654615
/baku/ads/salam-buzovna-qsbsind-cox-gozl-v-qdim-razid-id-83654615
/user/11272448
/user/11272448
/baku/ads/memar-ecemi-xutor-kucesi-1-otaqli-menzil-kiraye-verilir-meniml